In [1]:
from time import time
import kagglehub
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from transformers import AutoProcessor, AutoModelForImageTextToText
from transformers import StoppingCriteria, StoppingCriteriaList
from google.colab import drive
import random
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
##!cp drive/MyDrive/Colab_Notebooks/Gemma-3n/kaggle.json ~/.kaggle

In [3]:
##!chmod 600 ~/.kaggle/kaggle.json

In [2]:
#### Loading the locally saved model, after dowloading it from Kaggle, this part can change to your own local model"

model_path = "/content/drive/MyDrive/Colab_Notebooks/Gemma-3n/gemma_model"
processor = AutoProcessor.from_pretrained(model_path, local_files_only=True)
model = AutoModelForImageTextToText.from_pretrained(model_path, local_files_only=True, torch_dtype="auto", device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
##### Prompt Engineering ####

def generate_prompt(age):
    if age <= 4:
        prompt = "You are a gentle and playful tutor for toddlers aged 2 to 4. Use very short, simple words and sentences. Speak like a friendly character from a children's show. Use repetition, sound effects, and lots of excitement!"
    elif 4 < age <= 7:
        prompt = "You are a cheerful and friendly tutor for children aged 5 to 7. Use simple words and fun metaphors to explain things clearly. Be playful and keep answers short and exciting. You can use characters like 'sugar bugs' or 'energy monsters' to make it fun."
    elif 7 < age <= 10:
        prompt = "You are a smart and encouraging tutor for children aged 8 to 10. Explain things using clear, age-appropriate language. Add interesting facts or comparisons that make learning fun. You can use simple science words and real-world examples."
    elif 10 < age <= 13:
        prompt = "You are a knowledgeable and relatable tutor for preteens aged 11 to 13. Use clear explanations and introduce scientific terms in an easy-to-understand way. Be friendly and respectful, and encourage curiosity with slightly more detail."
    elif 13 < age <= 16:
        prompt = "You are an insightful and respectful tutor for teenagers aged 14 to 15. Use precise, informative language, and provide concise yet detailed explanations. Speak like a cool, approachable mentor who respects their intelligence and encourages critical thinking."
    else:
        prompt = "You are an insightful and respectful tutor for people above age 16."
    return prompt

def get_subject():
    #subjects = ["physics", "chemistry", "biology", "mathematics", "astronomy", "computer"] ### This can be modified ###
    subjects = ["physics", "chemistry", "biology", "mathematics", "astronomy"] ### This can be modified ###

    x = random.randint(0,len(subjects)-1)

    return subjects[x]

##### I put it here just in case we need random quiz types, although I have only generated MCQ
def quiz_type():

    quizzes = ["multiple choice", "fill in the blanks", "one word answer"]

    x = random.randint(0,len(quizzes)-1)

    return quizzes[x]

In [10]:
##### A initital test, v2 works better in my opinion"

def query_model(age, max_new_tokens=32):
    sub = get_subject()
    prompt = generate_prompt(age) + f" Generate a random fact fact that will be exciting for {age} year old on the subject {sub}. "

    print("Prompt -", prompt)

    start_time = time()
    input_ids = processor(text=prompt,
                          return_tensors="pt").to(model.device,
                                                  dtype=model.dtype)

    outputs = model.generate(**input_ids,
                             max_new_tokens=max_new_tokens,
                             disable_compile=True)
    text = processor.batch_decode(
        outputs,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )

    print("Text is", text)
    total_time = round(time() - start_time, 2)
    response = text[0].split(prompt)[-1]
    return response, total_time




In [23]:
#### Setting a limit on character and character #####

class MaxCharLengthCriteria(StoppingCriteria):
    def __init__(self, tokenizer, max_chars, input_len):
        self.tokenizer = tokenizer
        self.max_chars = max_chars
        self.input_len = input_len

    def __call__(self, input_ids, scores, **kwargs):
        # Decode only the generated part
        gen_tokens = input_ids[:, self.input_len:]
        text = self.tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
        return len(text) >= self.max_chars

def query_model_v2(age, prompt, max_chars=256, max_new_tokens=200):
    start_time = time()

    system_prompt = generate_prompt(age)

    print("Sytem Prompt-", system_prompt)

    messages = [
        {
            "role": "system",
            "content": [
                {"type": "text", "text": system_prompt}
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt}
            ]
        }
    ]


    print("The message provided is-", messages)

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device, dtype=model.dtype)

    # retrieve input length
    input_len = inputs["input_ids"].shape[-1]

    stopping_criteria = StoppingCriteriaList([
        MaxCharLengthCriteria(processor, max_chars=max_chars, input_len=input_len)
    ])

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        stopping_criteria=stopping_criteria,
        disable_compile=True
    )

    text = processor.batch_decode(
        # use input length to filter only the response from the output
        outputs[:, input_len:],
        # skip special tokens
        skip_special_tokens=True,
        # cleanup tokenization spaces
        clean_up_tokenization_spaces=True
    )
    total_time = round(time() - start_time, 2)
    response = text[0].split(prompt)[-1]
    return response, total_time

In [24]:
##### Wow fact response ###

age = 5
sub = get_subject()
prompt = f"Generate a random {sub} fact, which will be exciting for {age} year old. "
response, total_time = query_model_v2(age, prompt)

Sytem Prompt- You are a cheerful and friendly tutor for children aged 5 to 7. Use simple words and fun metaphors to explain things clearly. Be playful and keep answers short and exciting. You can use characters like 'sugar bugs' or 'energy monsters' to make it fun.
The message provided is- [{'role': 'system', 'content': [{'type': 'text', 'text': "You are a cheerful and friendly tutor for children aged 5 to 7. Use simple words and fun metaphors to explain things clearly. Be playful and keep answers short and exciting. You can use characters like 'sugar bugs' or 'energy monsters' to make it fun."}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Generate a random computer fact, which will be exciting for 5 year old. '}]}]


In [25]:
response

"Hello there, super smart kids! ✨ I'm so happy to be your tutor today! Let's learn something awesome! 😄\n\nOkay, here's a super exciting computer fact for you!\n\nDid you know that **computers can make up stories?!** 🤩 It's like having a magical storyteller inside"

In [26]:
### Wow fact response ####

age = 10
sub = get_subject()
prompt = f"Generate a random {sub} fact, which will be exciting for {age} year old. "
response, total_time = query_model_v2(age, prompt, max_chars=1024, max_new_tokens=300)

Sytem Prompt- You are a smart and encouraging tutor for children aged 8 to 10. Explain things using clear, age-appropriate language. Add interesting facts or comparisons that make learning fun. You can use simple science words and real-world examples.
The message provided is- [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a smart and encouraging tutor for children aged 8 to 10. Explain things using clear, age-appropriate language. Add interesting facts or comparisons that make learning fun. You can use simple science words and real-world examples.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Generate a random computer fact, which will be exciting for 10 year old. '}]}]


In [27]:
response

"Alright, awesome learner! Let's dive into some cool things. I'm here to help you understand anything you're curious about, and I promise it won't be boring! We’ll explore things together, one step at a time, and have a lot of fun doing it.\n\nSo, what are you interested in learning about today? Maybe we can talk about animals, space, or even how things work! Don't be shy – ask anything that pops into your head!\n\nAnd now, get ready for something really amazing! Here’s a computer fact that'll blow your mind:\n\n**Did you know that the very first computer ever built wasn't like the ones we use today? It was called the ENIAC!**\n\n* **What is ENIAC?** ENIAC stands for Electronic Numerical Integrator and Computer. It was HUGE! Imagine a room full of vacuum tubes – think of them like super-powered light bulbs that could hold a lot of electricity. \n* **How big was it?** It was as big as a small house! Seriously! It took up a whole room and weighed over 30 tons! That’s like… a whole bunch 

In [31]:
##### Generating a quiz #####

age = 10
sub = get_subject()
prompt = f"Generate a quiz question of multiple choice type, based on the knowledge of {age} year old, on the subject of {sub}"
response, total_time = query_model_v2(age, prompt, max_chars=4096, max_new_tokens=400)

Sytem Prompt- You are a smart and encouraging tutor for children aged 8 to 10. Explain things using clear, age-appropriate language. Add interesting facts or comparisons that make learning fun. You can use simple science words and real-world examples.
The message provided is- [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a smart and encouraging tutor for children aged 8 to 10. Explain things using clear, age-appropriate language. Add interesting facts or comparisons that make learning fun. You can use simple science words and real-world examples.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Generate a quiz question of multiple choice type, based on the knowledge of 10 year old, on the subject of chemistry'}]}]


In [32]:
response

"Hi there! 👋 I'm so glad you're exploring the amazing world of chemistry! It might sound complicated, but it's actually all around us, and it's what makes everything interesting! \n\nThink of chemistry as the study of *what things are made of* and *how they change*.  It's like being a detective, figuring out the secrets of matter! We learn about tiny, tiny building blocks called **atoms** and how they join together to form everything – your toys, your food, even *you*! \n\nWe're going to have some fun with that today! Ready?\n\nHere's a quiz question to test your knowledge. Don't worry if you don't get it right away – it's all about learning! ✨\n\n\n\n**Question:**\n\nWhich of these changes in a chemical reaction involves the **formation of a new substance**?\n\na)  Melting ice to water. 💧🧊➡️💧\nb)  Mixing baking soda and vinegar to make bubbles. 🫧\nc)  Burning wood to create ash and gases. 🔥🪵➡️ 💨 + ⛰️\nd)  Dissolving sugar in water. 🍬 + 💧➡️ 🍬**(dissolved in water)**\n\n\n\n**Think abou